In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
df = pd.read_excel("dav_wetzlar_results_2025_final.xlsx")

In [4]:
df.head()

,rank,name,team,category,discipline,event_name,location,date_start,date_end,url
0,23,Ole KÜSTER,Wetzlar,U13 m,Boulder,Offene Hessische Landesmeisterschaft Bouldern ...,Darmstadt (GER) – Boulderhaus Darmstadt,2025-02-01,2025-02-02,https://dav.results.info/event/1374/general/bo...
1,11,Inara GEBHARD,Wetzlar,U13 w,Boulder,Offene Hessische Landesmeisterschaft Bouldern ...,Darmstadt (GER) – Boulderhaus Darmstadt,2025-02-01,2025-02-02,https://dav.results.info/event/1374/general/bo...
2,7,Johanna OTTOFÜLLING,Wetzlar,U15 w,Boulder,Offene Hessische Landesmeisterschaft Bouldern ...,Darmstadt (GER) – Boulderhaus Darmstadt,2025-02-01,2025-02-02,https://dav.results.info/event/1374/general/bo...
3,7,Lasse KÜSTER,Wetzlar,U17 m,Boulder,Offene Hessische Landesmeisterschaft Bouldern ...,Darmstadt (GER) – Boulderhaus Darmstadt,2025-02-01,2025-02-02,https://dav.results.info/event/1374/general/bo...
4,18,Hendrik BRAUN,Wetzlar,U17 m,Boulder,Offene Hessische Landesmeisterschaft Bouldern ...,Darmstadt (GER) – Boulderhaus Darmstadt,2025-02-01,2025-02-02,https://dav.results.info/event/1374/general/bo...


In [6]:
# Ensure rank is numeric (some might be empty or non-numeric)
df["rank"] = pd.to_numeric(df["rank"], errors="coerce")

In [7]:
def analyze_athlete(df, name):
    """Return summary stats for one athlete by name (case-insensitive)."""
    subset = df[df["name"].str.lower() == name.lower()].copy()
    if subset.empty:
        return f"No records found for {name}"

    # Number of competitions
    num_comps = subset["event_name"].nunique()

    # Average rank (ignoring NaN)
    avg_rank = subset["rank"].mean()

    # Top 3 best results (lowest ranks)
    top3 = subset.nsmallest(3, "rank")[["event_name", "rank", "category", "date_start"]]

    summary = {
        "name": name,
        "competitions": num_comps,
        "average_rank": round(avg_rank, 2),
        "best_results": top3.to_dict("records")
    }
    return summary

In [13]:
athlete_stats = pd.DataFrame()

for athlete in df["name"].unique():
    result = analyze_athlete(df, athlete)
    athlete_stats = athlete_stats.concat(result, ignore_index=True)
athlete_stats.head(10)


AttributeError: 'DataFrame' object has no attribute 'concat'

In [21]:
summary_df = (
    df.groupby("name", as_index=False)
      .agg(
          competitions=("event_name", "nunique"),
          average_rank=("rank", "mean"),
          best_rank=("rank", "min"),
        second_best_rank=("rank", lambda x: x.nsmallest(2).max() if len(x) >= 2 else None),
            third_best_rank=("rank", lambda x: x.nsmallest(3).max() if len(x) >= 3 else None),
      )
      .sort_values("average_rank")
)

In [22]:
summary_df

,name,competitions,average_rank,best_rank,second_best_rank,third_best_rank
3,Frida PRZIBILLA,3,1.000000,1,1.0,1.0
10,Klara OTTOFÜLLING,4,5.000000,1,5.0,6.0
13,Lasse KÜSTER,7,6.714286,4,4.0,6.0
2,Erik POLLAK,11,6.750000,1,1.0,2.0
6,Inara GEBHARD,14,7.071429,1,2.0,4.0
12,Kristina IVANOVA,5,7.200000,1,5.0,8.0
14,Milla SCHICK,3,8.666667,8,9.0,9.0
1,Ben RAASCH,3,10.000000,9,9.0,12.0
15,Ole KÜSTER,9,10.333333,6,6.0,8.0
9,Jule BREUER,3,11.166667,3,4.0,9.0


In [26]:
for year in range(2025, 2026):
    df = pd.read_excel(f"dav_wetzlar_results_{year}_final.xlsx")
    summary_df = (
    df.groupby("name", as_index=False)
      .agg(
          competitions=("event_name", "nunique"),
          average_rank=("rank", "mean"),
          best_rank=("rank", "min"),
        second_best_rank=("rank", lambda x: x.nsmallest(2).max() if len(x) >= 2 else None),
            third_best_rank=("rank", lambda x: x.nsmallest(3).max() if len(x) >= 3 else None),
      )
      .sort_values("average_rank")
    )
    summary_df.to_excel(f"dav_wetzlar_results_{year}_summary.xlsx")
